---
title : "Overstromingsrisico scenarios"
execute:
    output: asis
---

In [30]:
from pathlib import Path

from toolbox_continu_inzicht.base.config import Config
from toolbox_continu_inzicht.base.data_adapter import DataAdapter

In [31]:
path = Path.cwd() / "data_sets" / "7.flood_scenarios"
config = Config(config_path=path / "test_calculate_flood_scenario_probability.yaml")
config.lees_config()
data_adapter = DataAdapter(config=config)

In [32]:
from toolbox_continu_inzicht.flood_scenarios import CalculateFloodScenarioProbability

Voor het berekenen van overstromingsrisico scenarios moeten de faalkansen van vakken gecobineerd worden tot faalkansen die vergeleken kunnen worden met de kansen van overstromingsscenario's. Hierbij wordt gebruik gemaakt van de gecombineerde kans van het vak en deze worden gecombineerd naar traject delen. 

```mermaid
graph TD
    A1[Dijkvak/section] --> D[Dijktraject deel/segment]
    B1[Dijkvak/section] --> D
    B2[Dijkvak/section] --> D
    C1[Dijkvak/section] --> E[Dijktraject deel/segment]
    C2[Dijkvak/section] --> E
    E --> F
    D --> F[Overstromingskans gebied/Dikesystem]
```

Dit voorbeeld laat zien hoe dit gedaan kan worden met behulp van de `CalculateFloodScenarioProbability` stap in de TBCI toolbox.
Hiervoor is de gecombineerde faalkans per dijkvak nodig en een mapping van dijkvakken naar dijktraject delen van daar naar de overstromingsscenario gebieden.
```yaml
GlobalVariables:
    rootdir: "data_sets/7.flood_scenarios"
    moments: [-24,0,24,48]

DataAdapter:
    default_options:
        csv:
            sep: ","
    combined_failure_probability_data:
        type: csv
        file: "combined_failure_probability_data.csv"
    section_id_to_segment_id:
        type: csv
        file: "section_id_to_segment_id.csv"
```

In [33]:
calculate_flood_scenario_probability = CalculateFloodScenarioProbability(
    data_adapter=data_adapter
)
calculate_flood_scenario_probability.run(
    input=["combined_failure_probability_data", "section_id_to_segment_id"],
    output="segment_failure_probability",
)
# todo: segment to dikesystem

In [34]:
calculate_flood_scenario_probability.df_in_sections_to_segment

,section_id,segment_id,length
0,1,34003,2
1,3,34003,3
2,4,34003,1
3,5,34002,1
4,6,34002,1
5,7,34002,1
6,8,34002,1
7,9,34002,1
8,15,34002,2
9,16,34002,3


In [35]:
calculate_flood_scenario_probability.df_in_grouped_sections_failure_probability

,section_id,failure_probability
0,1,5.500000e-01
1,3,1.000000e-10
2,4,1.000000e-10
3,5,1.000000e-02
4,6,1.000000e-10
5,7,1.000000e-10
6,8,1.000000e-10
7,9,1.000000e-10
8,15,1.000000e-10
9,16,1.000000e-10


In [36]:
calculate_flood_scenario_probability.df_out

,segment_id,failure_probability
0,34003,0.183333
1,34002,0.000526


In [37]:
# na rekenen van segment 34003
failure = 1
for prob, length in zip([5.500000e-01, 1.000000e-10, 1.000000e-10], [2, 3, 1]):
    failure *= 1 - prob * length / 6
1 - failure

0.1833333333877778